### 同一站点每片WAFER取最近的时间; 最后算法的结果有0和NA的检查、处理

In [1]:
import warnings
import pandas as pd
import os
warnings.filterwarnings('ignore')
from scipy.stats import ttest_ind, mannwhitneyu, shapiro, levene

In [2]:
import pyspark.pandas as ps
from pyspark.sql.functions import pandas_udf, PandasUDFType, countDistinct, when, col, rank, lit

In [3]:
# spark集群模式
from pyspark.sql import SparkSession

os.environ['PYSPARK_PYTHON'] = '/usr/local/python-3.9.13/bin/python3'
spark = SparkSession.builder \
    .appName("pandas_udf") \
    .config('spark.sql.session.timeZone', 'Asia/Shanghai') \
    .config("spark.scheduler.mode", "FAIR") \
    .config('spark.driver.memory', '4g') \
    .config('spark.driver.cores', '8') \
    .config('spark.executor.memory', '4g') \
    .config('spark.executor.cores', '8') \
    .config('spark.cores.max', '8') \
    .config('spark.driver.host', '192.168.22.28') \
    .master("spark://192.168.12.47:7077,192.168.12.48:7077") \
    .getOrCreate()

In [4]:
df_run_pandas = pd.read_csv("D:/Jupyterfiles/晶合MVAFDC_general开发/MVAanlysisDevelop/defect_algorithm/defect_by_wafer_labeled.csv")
df_run_pandas

,WAFER_ID,OPER_NO,LOT_ID,RECIPE_KEY,RECIPE_ID,RANDOM_DEFECTS,INSPECTION_TIME,label
0,NA0299-02,5FP10,NA0299.000,3828393,EMNDE015FP10,6.0,2023-04-10 12:30:28,1
1,NA0301-13,5FP10,NA0301.000,3827258,EMNDE015FP10,3.0,2023-04-10 12:30:03,1
2,NA0303-02,5FP10,NA0303.000,3831272,EMNDE015FP10,6.0,2023-04-10 07:43:26,1
3,NA0304-24,5FP10,NA0304.000,3827258,EMNDE015FP10,9.0,2023-04-10 13:22:14,1
4,NA0305-02,5FP10,NA0305.000,3827258,EMNDE015FP10,11.0,2023-04-10 13:14:54,1
...,...,...,...,...,...,...,...,...
2235,NAZ541-15,1CG40,NAZ541.300,5766658,FYZK7011CG40S5,2262.0,2023-09-13 07:53:54,0
2236,NAZ541-15,1CZ10,NAZ541.300,5112663,FYZK7011CZ1007,809.0,2023-08-30 21:00:18,0
2237,NAZ541-15,1CG40,NAZ541.300,4769683,FYZK7011CG40,NaN,2023-08-27 23:52:52,0
2238,NAZ541-15,1CG20,NAZ541.300,4871798,FYZK7011CG20,NaN,2023-08-26 20:36:43,0


In [124]:
# df_run_pandas.loc[df_run_pandas['label'] == 0, 'RANDOM_DEFECTS'].tolist()

In [5]:
df_run = ps.from_pandas(df_run_pandas).to_spark()
df_run.count()

2240

In [6]:
grpby_list = ['OPER_NO']
grps = (df_run.groupBy(grpby_list)
                .agg(countDistinct('WAFER_ID').alias('wafer_count'),
                     countDistinct('WAFER_ID', when(df_run['label'] == 0, 1)).alias('good_num'),
                     countDistinct('WAFER_ID', when(df_run['label'] == 1, 1)).alias('bad_num'))
                .orderBy('bad_num', ascending=False))
grps.show()

+----------+-----------+--------+-------+
|   OPER_NO|wafer_count|good_num|bad_num|
+----------+-----------+--------+-------+
|     5FP10|         78|       0|     78|
|       EOL|         47|      19|     28|
|     HEFEI|         65|      44|     21|
|     1CG10|         29|      15|     14|
|     1CZ10|         26|      14|     12|
|BH90WA12S1|         31|      21|     10|
|     PVP10|         12|       3|      9|
|     2UG30|         16|       7|      9|
|     1CG40|         29|      21|      8|
|     BBP60|         14|       7|      7|
|     1GG30|         36|      29|      7|
|     GRP30|         20|      13|      7|
|     2UP30|         11|       5|      6|
|     1DG10|          8|       2|      6|
|     1FG10|         32|      26|      6|
|     2EG20|         16|      10|      6|
|     1CP20|          9|       3|      6|
|     2DG10|          9|       3|      6|
|     GRP40|         18|      12|      6|
|     3UG30|         15|       9|      6|
+----------+-----------+--------+-

-----------pandas

In [41]:
# oper, recipe = '5FP15', 'EMNH015FP222'
# df_specific = df_run.filter(f"OPER_NO == '{oper}' AND RECIPE_ID == '{recipe}'")
# df_specific.show()

oper = '1FP01'
df_specific = df_run.filter(f"OPER_NO == '{oper}'")
df_specific.show()

+---------+-------+----------+----------+------------+--------------+-------------------+-----+
| WAFER_ID|OPER_NO|    LOT_ID|RECIPE_KEY|   RECIPE_ID|RANDOM_DEFECTS|    INSPECTION_TIME|label|
+---------+-------+----------+----------+------------+--------------+-------------------+-----+
|NAZ048-18|  1FP01|NAZ048.120|   2769299|FYZF0011FP01|           1.0|2023-03-09 10:42:48|    1|
|NAZ170-05|  1FP01|NAZ170.000|   2728737|FYZF0011FP01|           9.0|2022-10-08 19:56:12|    0|
|NAZ173-07|  1FP01|NAZ173.010|   2827072|FYZF0011FP01|           8.0|2022-10-23 03:14:12|    0|
+---------+-------+----------+----------+------------+--------------+-------------------+-----+



In [42]:
df_specific_pandas = df_specific.toPandas()
df_specific_pandas

,WAFER_ID,OPER_NO,LOT_ID,RECIPE_KEY,RECIPE_ID,RANDOM_DEFECTS,INSPECTION_TIME,label
0,NAZ048-18,1FP01,NAZ048.120,2769299,FYZF0011FP01,1.0,2023-03-09 10:42:48,1
1,NAZ170-05,1FP01,NAZ170.000,2728737,FYZF0011FP01,9.0,2022-10-08 19:56:12,0
2,NAZ173-07,1FP01,NAZ173.010,2827072,FYZF0011FP01,8.0,2022-10-23 03:14:12,0


In [43]:
def get_model_result(df_run):

    good_wafers = df_run.loc[df_run['label'] == 0, 'RANDOM_DEFECTS'].tolist()
    bad_wafers = df_run.loc[df_run['label'] == 1, 'RANDOM_DEFECTS'].tolist()
    print("good_wafers:", good_wafers)
    print("bad_wafers", bad_wafers)

    if len(good_wafers) == 0 or len(bad_wafers) == 0:
        print("可能有一个为空")
        return pd.DataFrame()

    p_shapiro_good, p_shapiro_bad, p_levene = do_normality_tests(good_wafers, bad_wafers)
    print("p_shapiro_good, p_shapiro_bad, p_levene:", p_shapiro_good, p_shapiro_bad, p_levene)
    
    statistic, p_value = get_difference_results(good_wafers, bad_wafers, p_shapiro_good, p_shapiro_bad, p_levene)
    print("statistic, p_value:", statistic, p_value)

    importance_dict = {'statistic': statistic, 'importance': 1 - p_value}
    importance_dict.update({col_: df_run[col_].values[0] for col_ in grpby_list})
    importance_res = pd.DataFrame(importance_dict, index=[0])
    return importance_res

res = get_model_result(df_specific_pandas)
res

good_wafers: [9.0, 8.0]
bad_wafers [1.0]
p_shapiro_good, p_shapiro_bad, p_levene: 1.0 1.0 0.11611652351681559
equal_var=True
statistic, p_value: 25.98076211353316 1.3039667249712685e-05


,statistic,importance,OPER_NO
0,25.980762,0.999987,1FP01


----------------- 

In [ ]:
levene(good_wafers, bad_wafers)

In [148]:
# good_wafers = df_specific.filter(col("label") == 0).select("RANDOM_DEFECTS").rdd.flatMap(lambda x: x).collect()
# bad_wafers = df_specific.filter(col("label") == 1).select("RANDOM_DEFECTS").rdd.flatMap(lambda x: x).collect()

In [35]:
def extend_wafers(good_wafers, bad_wafers):
    if len(good_wafers) < 3:
        n = len(good_wafers)
        g_mean = np.mean(good_wafers)
        good_wafers.extend([g_mean] * (3 - n))

    if len(bad_wafers) < 3:
        n = len(bad_wafers)
        b_mean = np.mean(bad_wafers)
        bad_wafers.extend([b_mean] * (3 - n))
    return good_wafers, bad_wafers


def do_normality_tests(good_wafers, bad_wafers):
    
    good_wafers, bad_wafers = extend_wafers(good_wafers=good_wafers, bad_wafers=bad_wafers)
    
    # Shapiro-Wilk test: Normality Assumption
    _, p_shapiro_good = shapiro(good_wafers)
    _, p_shapiro_bad = shapiro(bad_wafers)

    # Levene's test: Homogeneity of Variance Assumption
    _, p_levene = levene(good_wafers, bad_wafers)
    return p_shapiro_good, p_shapiro_bad, p_levene

def get_difference_results(good_wafers, bad_wafers, p_shapiro_good, p_shapiro_bad, p_levene, alpha=0.05):
    
    good_wafers, bad_wafers = extend_wafers(good_wafers=good_wafers, bad_wafers=bad_wafers)
    
    if p_shapiro_good > alpha and p_shapiro_bad > alpha and p_levene > alpha:
        print("equal_var=True")
        statistic, p_value = ttest_ind(good_wafers, bad_wafers, equal_var=True)
        
    elif p_shapiro_good > alpha and p_shapiro_bad > alpha and p_levene < alpha:
        print("equal_var=False")
        statistic, p_value = ttest_ind(good_wafers, bad_wafers, equal_var=False)
        
    else:
        print("mannwhitneyu")
        statistic, p_value = mannwhitneyu(good_wafers, bad_wafers)
    return statistic, p_value

In [175]:
good_wafers = [1,2, 3]
bad_wafers = [3, 4, 0]
p_shapiro_good, p_shapiro_bad, p_levene = do_normality_tests(good_wafers, bad_wafers)
get_difference_results(good_wafers, bad_wafers, p_shapiro_good, p_shapiro_bad, p_levene)

(-0.25000000000000006, 0.8149020114591812)

In [177]:
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField
def fit_defect_model(df, grpby_list):
    struct_fields = [StructField(col_, StringType(), True) for col_ in grpby_list]
    struct_fields.extend([StructField("statistic", FloatType(), True),
                          StructField("importance", FloatType(), True)])
    schema_all = StructType(struct_fields)
    
    @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
    def get_model_result(df_run):

        good_wafers = df_run.loc[df_run['label'] == 0, 'RANDOM_DEFECTS'].tolist()
        bad_wafers = df_run.loc[df_run['label'] == 1, 'RANDOM_DEFECTS'].tolist()
        
        if len(good_wafers) == 0 or len(bad_wafers) == 0:
            return pd.DataFrame()
            
        p_shapiro_good, p_shapiro_bad, p_levene = do_normality_tests(good_wafers, bad_wafers)
        statistic, p_value = get_difference_results(good_wafers, bad_wafers, p_shapiro_good, p_shapiro_bad, p_levene)

        importance_dict = {'statistic': statistic, 'importance': 1 - p_value}
        importance_dict.update({col_: df_run[col_].values[0] for col_ in grpby_list})
        importance_res = pd.DataFrame(importance_dict, index=[0])
        return importance_res
    return df.groupby(grpby_list).apply(get_model_result)

In [178]:
res = fit_defect_model(df=df_run, grpby_list=['OPER_NO', 'RECIPE_ID'])
res.toPandas()

,OPER_NO,RECIPE_ID,statistic,importance
0,5FP10,EMNDE015FP10,0.0,0.982549
1,5FP10,EMNH015FP15,0.0,0.987879
2,5FP14,EMNH015FP222,9.0,0.923477
3,5FP15,EMNH015FP18,0.0,0.996865
4,5FP15,EMNH015FP222,3.0,0.342095


#### 检验

In [57]:
shapiro([1,2, 30])  #  Data must be at least length 3.

ShapiroResult(statistic=0.7758302092552185, pvalue=0.058016655848620036)

In [67]:
levene([1, 2, 9], [3, 200, 56])  # 方差齐性检验, 也最好都是三个

LeveneResult(statistic=2.2387189771872653, pvalue=0.2089273956185376)

#### equal_var=True的情况，必须其中一个的数量>=2

In [17]:
ttest_ind([1, 0], [3], equal_var=True)

TtestResult(statistic=-2.886751345948129, pvalue=0.21229561500965655, df=1.0)

In [23]:
ttest_ind([1], [3, 4], equal_var=True)

TtestResult(statistic=-2.886751345948129, pvalue=0.21229561500965655, df=1.0)

#### equal_var=False的情况，两个的数量都要>=2

In [25]:
ttest_ind([1, 0], [3, 5], equal_var=False)

TtestResult(statistic=-3.1304951684997055, pvalue=0.12904783554828336, df=1.4705882352941178)

#### mannwhitneyu检验，如果都只有一个值，结果永远是pvalue=1.0，无效, 最好两个以上

In [53]:
mannwhitneyu([-111196, -56], [200, 3])

MannwhitneyuResult(statistic=0.0, pvalue=0.3333333333333333)

In [52]:
# 进行 Mann-Whitney U 检验, P值越小越显著